<a href="https://colab.research.google.com/github/JDslay/LKA/blob/main/DCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
# some functions needed for the analysis

# All predictions in one tensor
@torch.no_grad()  # gradients not needed while evaluation / saves memory if turned off
def get_all_preds(model, loader):
    all_preds = torch.tensor([])
    for batch in loader:
        images, labels = batch
        preds = model(images)
        all_preds = torch.cat((all_preds, preds), dim=0)
    return all_preds

# Number of correct predictions
def get_nbr_correct_predicted(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [ ]:
# load data and prepare

train_set = torchvision.datasets.ImageFolder(
    root='./data/40_tobacco/train', transform=transforms.Compose([
        transforms.Resize((227, 227)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=10)  # getting batches out of dataset

In [ ]:
# load pretrained alexnet and prepare for finetuning

alexNetMod = models.alexnet(pretrained=True)

# freeze parameters
for param in alexNetMod.parameters():
    param.requires_grad = False

# change last fully connected layer with new and 10 outputs
alexNetMod.classifier[6] = nn.Linear(4096, 10)

In [ ]:
# TRAIN MODEL
optimizer = optim.RMSprop(alexNetMod.parameters(
), lr=0.0001, alpha=0.99, eps=1e-08, weight_decay=0.0005, momentum=0.9, centered=False)

# training loop
for epoch in range(15):
    total_loss = 0
    total_correct = 0
    for batch in train_loader:                  # Get Batch
        images, labels = batch
        preds = alexNetMod(images)                 # Pass Batch
        loss = F.cross_entropy(preds, labels)   # Calculate Loss
        optimizer.zero_grad()
        loss.backward()                         # Calculate Gradient
        optimizer.step()                        # Updating weights
        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    print('epoch:', epoch, 'total_correct:',
          total_correct, "loss:", total_loss)

print(total_correct/len(train_set))

In [ ]:
# Confusion Matrix
train_preds = get_all_preds(alexNetMod, train_loader)
cm = confusion_matrix(torch.tensor(train_set.targets), train_preds.argmax(dim=1))
names = ('ADVE', 'Email', 'Form', 'Letter', 'Memo',
         'News', 'Note', 'Report', 'Resume', 'Scientific')
plt.figure(figsize=(10, 10))
plot_confusion_matrix(cm, names)
plt.show()